In [1]:
import cv2, PIL, os, sys
import copy
import math
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
from PIL import Image

from cropimage import crop_single_image
from processing import Processor

%matplotlib nbagg
%load_ext autoreload
%autoreload 2

In [2]:

class WarpVideo:
    def __init__(self, frames_path: str, chessboard_vid_path: str, destination_frames_folder:str | None = None , chessboard_shape: tuple[int, int] = (6, 9)) -> None:
        self.frames_path: str = frames_path
        self.chessboard_vid_path: str = chessboard_vid_path
        self.chessboard_shape: tuple[int, int] = chessboard_shape
        self.destination_frames_folder: str | None = destination_frames_folder
        self.chessboard_images: list[np.ndarray] | None = None
        self.camera_matrix: np.ndarray | None = None
        self.dist_coeff: np.ndarray | None = None
        self.undistorted_frames: np.ndarray |list[np.ndarray] | None = None
        self.adjusted_matrix: np.ndarray | None = None
        self.adjusted_width: int | None = None
        self.adjusted_height: int | None = None
        self.corresp_imgnames: list[str] | None = None
        self.saved_img_names:list[str] | None = None
    
    @staticmethod
    def display_video(frames):
        for frame in frames:
            cv2.imshow("Frame", frame)
            cv2.waitKey(30)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cv2.destroyAllWindows()
    
    def __load_chessboard_images(self, every_x_frames:int = 20, overwrite: bool = False) -> list[np.ndarray]:
        char_frames = []
        for i, file in enumerate(os.listdir(self.chessboard_vid_path)):
            current_frame = -1
            charuco_vid_file_1 = f"{self.chessboard_vid_path}/{file}"
            video = cv2.VideoCapture(charuco_vid_file_1)
            grabbed, frame = video.read()
            while grabbed:
                current_frame += 1
                if not current_frame % every_x_frames:
                    char_frames.append(frame)
                grabbed, frame = video.read()
        print("number of pics loaded: ", len(char_frames))
        if overwrite:
            self.chessboard_images = char_frames
        return char_frames

    def find_intrinsic(self, overwrite:bool = False) -> tuple[np.ndarray, np.ndarray]:
        self.chessboard_images = self.__load_chessboard_images(overwrite=overwrite) if self.chessboard_images is None else self.chessboard_images
        
        copiedframes = copy.deepcopy(self.chessboard_images)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

        # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(4,7,0)
        objp = np.zeros((self.chessboard_shape[0] * self.chessboard_shape[1], 3), np.float32)
        objp[:, :2] = np.mgrid[0: self.chessboard_shape[0], 0: self.chessboard_shape[1]].T.reshape(-1, 2)
        
        size_of_chessboard_squares_mm = 30
        objp = objp * size_of_chessboard_squares_mm
        
        # Arrays to store object points and image points from all the images.
        objpoints = []  # 3d point in real world space
        imgpoints = []  # 2d points in image plane
        
        # Use the first frame to determine the frame size
        if len(copiedframes) > 0:
            frameHeight, frameWidth = copiedframes[0].shape[:2]
            frameSize = (frameWidth, frameHeight)
        else:
            raise ValueError("No frames in charucoFrames list")
    
        found = 0
        for idx, frame in enumerate(copiedframes):
            if frame is None:
                continue
            
            # Downscale the image by a factor of 5
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
            # Find the chess board corners
            ret, corners = cv2.findChessboardCorners(gray, self.chessboard_shape, None)
        
            if ret:
                found += 1
                objpoints.append(objp)
                corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
                imgpoints.append(corners2)
        
        if len(objpoints) == 0 or len(imgpoints) == 0:
            raise ValueError("No chessboard corners found in any of the frames.")
        print(f"Found chessboard corners in {found} frames.")
        
        ret, cameraMatrix, dist, _, _ = cv2.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

        if not ret:
            raise ValueError("Camera calibration failed.")
        print(f"Camera calibration succeeded. RME: {ret} pixels")
        self.camera_matrix, self.dist_coeff = cameraMatrix, dist if overwrite or (self.camera_matrix, self.dist_coeff) == (None, None) else None
        return cameraMatrix, dist
    
                
    def undistort_frames(self, cameraMatrix:np.ndarray|None = None, dist:np.ndarray|None=None, pathToFolder:str|None=None, return_imgs: bool = False, overwrite_existing:bool=True) -> None | list[np.ndarray]:
        if pathToFolder is None:
            pathToFolder = self.frames_path
            if pathToFolder is None:
                raise ValueError("No frames path specified.")
        if cameraMatrix is None:
            cameraMatrix = self.camera_matrix
            if cameraMatrix is None:
                raise ValueError("Camera matrix is empty.")
        if dist is None:
            dist = self.dist_coeff
            if dist is None:
                raise ValueError("Distance coefficient is empty.")
        undistorted_frames = []
        if self.undistorted_frames is None:
            self.undistorted_frames = []
        if self.saved_img_names is None:
            self.saved_img_names = []
        for filename in os.listdir(pathToFolder):
            frame_path = os.path.join(pathToFolder, filename)
            frame = cv2.imread(frame_path)
            h, w = frame.shape[:2]
            newCameraMatrix, _ = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (w, h), 1, (w, h))

            # Undistort the image
            undistorted = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
            if return_imgs:
                undistorted_frames.append(undistorted)
            if overwrite_existing:
                self.undistorted_frames.append(undistorted)
                self.saved_img_names.append(filename)
        if return_imgs:
            return undistorted_frames
        return None
                
            
    
    def findWarpPerspective(self, image_path, image_points, cornerpoints, image: np.ndarray | None = None):
        
        # Load the image
        if image is None:
            image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image at {image_path}")
            exit()
        # Create a figure and axis
        # Display the input image
        # ax.imshow(image)
        # plt.close()

        # Add dots for each coordinate
        # for point in image_points:
        #     ax.scatter(point[0], point[1], color='red', s=40)  # s is the size of the dot
        if len(image_points) != 4:
            print("You need to select exactly 4 points.")
            exit()
        # Convert points to numpy float32 format
        pts1 = np.float32(image_points)
        # Compute the width and height of the quadrilateral
        width_top = np.linalg.norm(pts1[0] - pts1[1])
        width_bottom = np.linalg.norm(pts1[2] - pts1[3])
        height_left = np.linalg.norm(pts1[0] - pts1[3])
        height_right = np.linalg.norm(pts1[1] - pts1[2])

        # Use the maximum of the widths and heights to define the square size
        max_width = max(int(width_top), int(width_bottom))
        max_height = max(int(height_left), int(height_right))
        square_size = max(max_width, max_height)

        # Define the destination points as a square with the calculated size
        pts2 = np.float32([
            [0, 0],
            [square_size - 1, 0],
            [square_size - 1, square_size - 1],
            [0, square_size - 1]
        ])
        # Get the perspective transform matrix
        matrix = cv2.getPerspectiveTransform(pts1, pts2)
        # Warp the entire image using the perspective transform matrix
        # To keep the whole image visible, let's compute the output bounds
        h, w = image.shape[:2]

        # Transform the four corners of the original image
        if cornerpoints is None or len(cornerpoints) == 0:
            corners_points = np.float32([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]])
        else:
            corners_points = np.float32(cornerpoints)
        
        transformed_corners = cv2.perspectiveTransform(corners_points[None, :, :], matrix)[0]
        # Find the bounding box of the transformed corners
        x_min, y_min = np.min(transformed_corners, axis=0).astype(int)
        x_max, y_max = np.max(transformed_corners, axis=0).astype(int)
        
        # Calculate the size of the new image
        new_width = x_max - x_min
        new_height = y_max - y_min

        # Create the translation matrix to shift the image to the positive coordinates
        translation_matrix = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])

        # Adjust the perspective transform matrix with the translation
        adjusted_matrix = translation_matrix @ matrix
        
        # Perform the warp with the adjusted matrix
        result = cv2.warpPerspective(image, adjusted_matrix, (new_width, new_height), flags=cv2.INTER_LINEAR,
                                    borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
        
        # Display the transformed image using matplotlib
        # plt.figure(figsize=(10, 10))
        # plt.imshow(result)
        # cv2.imwrite("../../Test Data/charuco_temp_folder/measurement_frames_formatted/inverted.jpg", cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        # plt.title("Perspective Transform Applied to Entire Image")
        # plt.show()
        impth = "../../Test Data/charuco_temp_folder/abcdef.jpg"
        abc, defg, _ = result.shape
        # print(result.shape)
        
        self.adjusted_matrix, self.adjusted_width, self.adjusted_height = adjusted_matrix, new_width, new_height
        
        # result_resized = cv2.resize(result, (defg // 4, abc// 4), interpolation=cv2.INTER_LINEAR)
        # cv2.imwrite(impth, result)
        # cv2.imshow("res", result_resized)
        # cv2.waitKey()
        # cv2.destroyAllWindows()
    
    def __warp_image(self, image):
        return cv2.warpPerspective(image, self.adjusted_matrix, (self.adjusted_width, self.adjusted_height), flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
        
    
    def generate_warped_images(self, camera_matrix:np.ndarray|None = None, dist_coeff:np.ndarray|None=None, path_to_write_vids:str|None=None, path_to_distorted_pics:str|None=None, path_to_undistorted_pics:str|None=None, filenames:list[str] | np.ndarray[str] |None=None):
        if camera_matrix is None:
            camera_matrix = self.camera_matrix
        if dist_coeff is None:
            dist_coeff = self.dist_coeff
        if dist_coeff is None or camera_matrix is None:
            self.dist_coeff = None
            self.camera_matrix = None
            self.find_intrinsic(overwrite=True)
        
        if path_to_write_vids is None:
            raise ValueError("No destination for warped videos found.")
        if path_to_distorted_pics is not None:
            undistorted_imgs = self.undistort_frames(pathToFolder=path_to_distorted_pics, return_imgs=True, overwrite_existing = False)
        if self.undistorted_frames is None:
            self.undistort_frames()
        else:
            undistorted_imgs = self.undistorted_frames
        if self.adjusted_matrix is None or self.adjusted_width is None or self.adjusted_height is None:
            raise ValueError("You need to find the warp matrix. Run self.findWarpPerspective first.")
        
        for i, frame in enumerate(self.undistorted_frames):
            img_name = f"{i}.jpg" if filenames is None else filenames[i]
            warped_image = self.__warp_image(frame)
            cv2.imwrite(f"{path_to_write_vids}/{img_name}", warped_image)
            
        
        

In [3]:
# warp_img_path = "../../Test Data/charuco_temp_folder/testtest"
unwarped_undistorted_images_folder = "../../Test Data/charuco_temp_folder/testwarping/e_9_percent"
warped_destination_folder = "../../Test Data/charuco_temp_folder/testwarping/destination"
x = np.float32([[1.44058342 * 10 ** 3, 0.00000000, 9.49703258 * 10 ** 2],
 [0.00000000, 1.43643171 * 10 ** 3, 5.66369243 * 10 ** 2],
 [0.00000000, 0.00000000, 1.00000000]])
y = np.float32([[1.92756759 * 10 ** -1, -7.20995760 * 10 ** -1,  3.34726484 * 10 ** -3, -5.61250479 * 10 ** -4,
   7.99993126 * 10 ** -1]])

In [1]:
obj = WarpVideo(unwarped_undistorted_images_folder, "../../Test Data/charuco_temp_folder/intrinsic_vid")
obj.camera_matrix = x
obj.dist_coeff = y
# obj.find_intrinsic()

warp_img_path = "../../Test Data/charuco_temp_folder/chessboard_pic/middle_frame.jpg"
# image_points = [
#     [1070, 315],
#     [1557, 395],
#     [1677, 654],
#     [997, 532]
# ]
image_points_undistorted = [
    [1071, 317],
    [1557, 397],
    [1676, 655],
    [997, 535]
]

corners = np.float32([
    [276,64],
    [1836,438],
    [1769,1038],
    [100,974]
])
obj.findWarpPerspective(warp_img_path, image_points_undistorted, corners)

NameError: name 'WarpVideo' is not defined

In [118]:
# obj.generate_warped_images(path_to_write_vids=warped_destination_folder)

In [114]:
# # undistort chessboard image, and manually find image points and corners.
# chessboard_img_folder = "../../Test Data/charuco_temp_folder/testwarping/chessboard_img"
# chessboard_dest = "../../Test Data/charuco_temp_folder/testwarping/chessboard_undistorted"
# chessboard_imgs = obj.undistort_frames(return_imgs=True, overwrite_existing=False, pathToFolder=chessboard_img_folder)
# cv2.imwrite(f"{chessboard_dest}/undistorted_chessboard.jpg", chessboard_imgs[0])

True

In [323]:
# obj.frames_path = warp_img_path

In [338]:
# obj.undistort_frames()

-3
-2
-1
0


In [5]:
path_to_vids_folder = "../../Test Data/charuco_temp_folder/lange video 1 deel"
path_write_all_frames = "../../Test Data/charuco_temp_folder/measurement_frames_formatted"
start_time = datetime.datetime(2024, 6, 7, 15, 1, 59, 270000)
snippets_test = np.array([
    [datetime.datetime(2024, 6, 7, 15, 2, 57), datetime.datetime(2024, 6, 7, 15, 2, 58)],
    [datetime.datetime(2024, 6, 7, 15, 3, 22), datetime.datetime(2024, 6, 7, 15, 3, 24)],
    [datetime.datetime(2024, 6, 7, 15, 3, 50), datetime.datetime(2024, 6, 7, 15, 3, 52)],
    [datetime.datetime(2024, 6, 7, 15, 4, 11), datetime.datetime(2024, 6, 7, 15, 4, 12)],
    [datetime.datetime(2024, 6, 7, 15, 4, 31), datetime.datetime(2024, 6, 7, 15, 4, 31, 500000)],
    [datetime.datetime(2024, 6, 7, 15, 4, 50), datetime.datetime(2024, 6, 7, 15, 4, 51)]])
snippets_step_wind = np.array([
    [datetime.datetime(2024, 6, 7, 15, 2, 45), datetime.datetime(2024, 6, 7, 15, 3, 0)],
    [datetime.datetime(2024, 6, 7, 15, 3, 4), datetime.datetime(2024, 6, 7, 15, 3, 30)],
    [datetime.datetime(2024, 6, 7, 15, 3, 38), datetime.datetime(2024, 6, 7, 15, 3, 52)],
    [datetime.datetime(2024, 6, 7, 15, 4, 0), datetime.datetime(2024, 6, 7, 15, 4, 13)],
    [datetime.datetime(2024, 6, 7, 15, 4, 20), datetime.datetime(2024, 6, 7, 15, 4, 32)],
    [datetime.datetime(2024, 6, 7, 15,4, 36), datetime.datetime(2024, 6, 7, 15, 4, 54)]])
folderabspath = "../../Test Data/THE REAL DEAL/test_to_be_processed"
background_img_file = "../../Test Data/IMG_2378_goed/200.jpg"

In [89]:
# video = cv2.VideoCapture(f"{path_to_vids_folder}/{os.listdir(path_to_vids_folder)[0]}")
# video.set(cv2.CAP_PROP_POS_FRAMES, 200)
# grabbed, frame = video.read()
# video.release()
# frame = cv2.resize(frame, (frame.shape[1], frame.shape[0]), cv2.INTER_AREA)
# cv2.imshow("frame", frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# for file in os.listdir(folderabspath):
#     cv2.imwrite(f"{folderabspath}/{file}/000000000000_background_frame.jpg", frame)

In [33]:
for file in os.listdir(folder_to_be_processed):
    cv2.imwrite(f"{folder_to_be_processed}/{file}/00000000000000_background_frame.jpg", cv2.imread(background_img_file))

In [6]:

def create_folders(folder_destination:str|None = None):
    if folder_destination is None:
        folderabspath = "../../Test Data/THE REAL DEAL/Unprocessed Frame Pairs"
    else:
        folderabspath = folder_destination
    letters = ["a", "b", "c", "d", "e", "f"]
    for idx, i in enumerate(range(5, 11)):
        folder_path = os.path.join(folderabspath, f"{letters[idx]}_{i}_percent")
        os.makedirs(folder_path, exist_ok=True)
        
    

In [16]:
output_folder = "../../Test Data/THE REAL DEAL/Background Removed and Cropped"
test_output = "../../Test Data/THE REAL DEAL/test_output/a_5_percent"
test_folder_to_be_processed = "../../Test Data/THE REAL DEAL/test_to_be_processed/a_5_percent"
# create_folders(test_output)
# first mask & remove background
processor = Processor(path=test_folder_to_be_processed, df="jpg")
images_in_folder = os.listdir(test_folder_to_be_processed)
# images_in_background_folder = os.listdir(background_folder)
background_reference = cv2.imread(f"{test_folder_to_be_processed}/{images_in_folder[0]}", cv2.IMREAD_GRAYSCALE)
test_frame = cv2.imread(f"{test_folder_to_be_processed}/{images_in_folder[1]}", cv2.IMREAD_GRAYSCALE)
# test_frame = background_reference
background_image_display = cv2.resize(background_reference.copy(), (background_reference.shape[1]//2, background_reference.shape[0]//2), cv2.INTER_AREA)
# cv2.imshow("background_image", background_image_display)
test_frame_display = cv2.resize(test_frame.copy(), (test_frame.shape[1]//2, test_frame.shape[0]//2), cv2.INTER_AREA)
denoised_background_img = processor.denoise(background_reference)
processor.reference = denoised_background_img
# processor.reference = background_reference
denoised_test_img = processor.denoise(test_frame)
masked_test_img = processor.mask(denoised_test_img)
warped_test_img = cv2.warpPerspective(masked_test_img, obj.adjusted_matrix, (obj.adjusted_width, obj.adjusted_height), flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
cropped_test_img = crop_single_image(warped_test_img, crop_corner_points=(1295, 1581, 1745, 2185))
warped_test_frame_display = cv2.resize(warped_test_img.copy(), (warped_test_img.shape[1]//2, warped_test_img.shape[0]//2), cv2.INTER_AREA)
cropped_test_frame_display = cv2.resize(cropped_test_img.copy(), (cropped_test_img.shape[1]//2, cropped_test_img.shape[0]//2), cv2.INTER_AREA)
# cv2.imshow("masked_img", warped_test_frame_display)
# cv2.imshow("test_frame", test_frame_display)
cv2.imshow("cropped_img", cropped_test_frame_display)
cv2.waitKey(0)
cv2.destroyAllWindows()
# crop_images(test_folder_to_be_processed, test_output)
print(cropped_test_img.shape)

(450, 604)


In [11]:
cv2.imwrite("../../Test Data/THE REAL DEAL/test_output/a_5_percent/PAINT.jpg", warped_test_img)

True

In [31]:
def do_everything(source_folder, destination_folder):
    background = None
    processor = Processor(path=source_folder, df="jpg")
    for i, file in enumerate(os.listdir(source_folder)):
        image = cv2.imread(f"{source_folder}/{file}", cv2.IMREAD_GRAYSCALE)
        if background is None:
            background = processor.denoise(image)
            processor.reference = background
            continue
        # print(i)
        denoised_image = processor.denoise(image)
        masked_image = processor.mask(denoised_image)
        warped_image = cv2.warpPerspective(masked_image, obj.adjusted_matrix, (obj.adjusted_width, obj.adjusted_height), flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
        cropped_image = crop_single_image(warped_image, crop_corner_points=(1295, 1581, 1745, 2185))
        cv2.imwrite(f"{destination_folder}/{file}", cropped_image)
        # print("img_1_done")
        # print(f"{destination_folder}/{file}")
        

In [35]:
folder_to_be_processed = "../../Test Data/charuco_temp_folder/measurement_frames_formatted"
a_output_folder = "../../Test Data/THE REAL DEAL/Background Removed and Cropped"
output_folders = os.listdir(a_output_folder)
for i, folder in enumerate(os.listdir(folder_to_be_processed)):
    do_everything(f"{folder_to_be_processed}/{folder}", f"{a_output_folder}/{output_folders[i]}")

In [34]:
create_folders(a_output_folder)

In [11]:
def get_video_snippets(start_time: datetime.datetime,  time_snippets: list[datetime.datetime] | np.ndarray[datetime.time], backwards_frames: int, frames_per_real_second: int, pairs_per_second: int, path_write_frames_folder: str, path_to_vids_folder: str):
    create_folders()
    time_per_frame =  1 / 30  # seconds in slowed video time
    # snippet_duration = 10 * frames_per_real_second
    snippet_duration = backwards_frames
    frame_count_per_vid = []
    frame_locs = []
    times = []
    folders_to_write = os.listdir(path_write_frames_folder)
    video_names = os.listdir(path_to_vids_folder)
    for video_file in os.listdir(path_to_vids_folder):
        # print(path_to_vids_folder)
        # print(video_file)
        video = cv2.VideoCapture(os.path.join(path_to_vids_folder, video_file))
        frame_count_per_vid.append(video.get(cv2.CAP_PROP_FRAME_COUNT))
        video.release()
    frame_count_per_vid = np.cumsum(np.array(frame_count_per_vid)).astype(int)
    for ii, (start_snippet, end_snippet) in enumerate(time_snippets):
        current_frame_number = round((start_snippet - start_time).total_seconds() * frames_per_real_second)
        frame_snip_locs = []
        times_per_snip = []
        tot_frames_snip = round((end_snippet - start_snippet).total_seconds() * frames_per_real_second)
        # print(tot_frames_snip)
        for jj in range(0, round(tot_frames_snip / frames_per_real_second * pairs_per_second)):
            frame_snip_locs.append(current_frame_number + round(jj * frames_per_real_second / pairs_per_second)) 
            frame_snip_locs.append(current_frame_number + round(jj * frames_per_real_second / pairs_per_second) + 1)
            times_per_snip.append((start_snippet + datetime.timedelta(0, jj / pairs_per_second)).strftime("%d_%m_%Y_%H_%M_%S_%f")[:-3])
            times_per_snip.append((start_snippet + datetime.timedelta(0, jj / pairs_per_second + 1 / frames_per_real_second)).strftime("%d_%m_%Y_%H_%M_%S_%f")[:-3])
        frame_locs.append(frame_snip_locs)
        times.append(times_per_snip)
        
    # print(times)
    # print([len(x) for x in times])
    print(frame_locs)
    print(frame_count_per_vid)
    print([len(x) for x in frame_locs])
    
    singlevideo = False
    if len(video_names) == 1:
        singlevideo = True
    # VIDEOPART
    if singlevideo:
        video = cv2.VideoCapture(os.path.join(path_to_vids_folder, video_names[0]))
    for i, snip_frames_list in enumerate(frame_locs):
        for j, current_frame_number in enumerate(snip_frames_list):
            for k, count in enumerate(frame_count_per_vid):
                if singlevideo:
                    video.set(cv2.CAP_PROP_POS_FRAMES, current_frame_number)
                    grabbed, frame = video.read()
                    if not grabbed:
                        print(f"frame_location: a{frame_location}")
                        print(f"video size: {frame_count_per_vid[k]}")
                        print(f"count: a{count}")
                        print(f"current_frame_number: {current_frame_number}")
                        return
                    img_path = f"{path_write_frames_folder}/{folders_to_write[i]}/{times[i][j]}.jpg"
                    cv2.imwrite(img_path, frame)
                    continue
                    
                
                if current_frame_number > count:
                    continue
                frame_location = current_frame_number - frame_count_per_vid[k-1] if k > 0 else current_frame_number
                # print(f"current_frame_number: {current_frame_number},\nframe_location: {frame_location}")
                # print(f"frame_count_per_vid[k-1]: {frame_count_per_vid[k-1]}")
                video = cv2.VideoCapture(os.path.join(path_to_vids_folder, video_names[k]))
                video.set(cv2.CAP_PROP_POS_FRAMES, frame_location)
                grabbed, frame = video.read()
                if not grabbed:
                    print(f"frame_location: a{frame_location}")
                    print(f"video size: {frame_count_per_vid[k]}")
                    print(f"count: a{count}")
                    print(f"current_frame_number: {current_frame_number}")
                    return
                img_path = f"{path_write_frames_folder}/{folders_to_write[i]}/{times[i][j]}.jpg"
                # print(f"folder: {i}")
                # print(f"video: {k}")
                # print(f"frame_location: {frame_location}")
                # print(f"count: {current_frame_number}")
                cv2.imwrite(img_path, frame)
                video.release()
                break
            continue
        print(f"Folder {i} filled")
    if singlevideo:
        video.release()
    return frame_locs, times, frame_count_per_vid


In [28]:
backwards_frames = 1
framerate_real = 120
pairs_per_second = 6
framelocs, timees, countpervid = get_video_snippets(start_time=start_time,
                   time_snippets=snippets_step_wind,
                   backwards_frames=backwards_frames,
                   frames_per_real_second=framerate_real, 
                   pairs_per_second=pairs_per_second,
                   path_write_frames_folder=path_write_all_frames,
                   path_to_vids_folder=path_to_vids_folder)

[[5488, 5489, 5508, 5509, 5528, 5529, 5548, 5549, 5568, 5569, 5588, 5589, 5608, 5609, 5628, 5629, 5648, 5649, 5668, 5669, 5688, 5689, 5708, 5709, 5728, 5729, 5748, 5749, 5768, 5769, 5788, 5789, 5808, 5809, 5828, 5829, 5848, 5849, 5868, 5869, 5888, 5889, 5908, 5909, 5928, 5929, 5948, 5949, 5968, 5969, 5988, 5989, 6008, 6009, 6028, 6029, 6048, 6049, 6068, 6069, 6088, 6089, 6108, 6109, 6128, 6129, 6148, 6149, 6168, 6169, 6188, 6189, 6208, 6209, 6228, 6229, 6248, 6249, 6268, 6269, 6288, 6289, 6308, 6309, 6328, 6329, 6348, 6349, 6368, 6369, 6388, 6389, 6408, 6409, 6428, 6429, 6448, 6449, 6468, 6469, 6488, 6489, 6508, 6509, 6528, 6529, 6548, 6549, 6568, 6569, 6588, 6589, 6608, 6609, 6628, 6629, 6648, 6649, 6668, 6669, 6688, 6689, 6708, 6709, 6728, 6729, 6748, 6749, 6768, 6769, 6788, 6789, 6808, 6809, 6828, 6829, 6848, 6849, 6868, 6869, 6888, 6889, 6908, 6909, 6928, 6929, 6948, 6949, 6968, 6969, 6988, 6989, 7008, 7009, 7028, 7029, 7048, 7049, 7068, 7069, 7088, 7089, 7108, 7109, 7128, 7129, 71

In [122]:
print(sum([204, 312, 168, 156, 144, 216]))

1200


In [ ]:
    # print(f"frame_number_abs: {current_frame_number}")
    # 
    #     # print(j)
    # 
    #     # print("frame_loc_rel ", frame_location_start)
    #     video = cv2.VideoCapture(os.path.join(path_to_vids_folder, os.listdir(path_to_vids_folder)[j]))
    #     if frame_location_start - snippet_duration + 1 < 0:  # assumed j > 0
    #         # print("XX")
    #         # print("frame_loc_start ", frame_location_start)
    #         start_prev_vid = frame_count_per_vid[j-1] - frame_count_per_vid[j-2] + frame_location_start - snippet_duration
    #         # print("start_prev_vid", start_prev_vid)
    #         video_prev = cv2.VideoCapture(os.path.join(path_to_vids_folder, os.listdir(path_to_vids_folder)[j-1]))
    #         # print(video_prev.get(cv2.CAP_PROP_FRAME_COUNT))
    #         video_prev.set(cv2.CAP_PROP_POS_FRAMES, start_prev_vid - 1)
    #         counter = -1
    #         grabbed, frame = video_prev.read()
    #         # print(grabbed)
    #         while grabbed:
    #             counter += 1
    #             # print(os.path.join(path_write_frames_folder, folders_to_write[i], f"{start_time+datetime.timedelta(0,(frame_number - snippet_duration + counter)/time_per_frame)}.jpg"))
    #             current_time_str = (current_time + datetime.timedelta(0, (-snippet_duration + counter) / frames_per_real_second)).strftime("%d_%m_%Y_%H_%M_%S_%f")[:-3]
    #             img_path = f"{path_write_frames_folder}/{folders_to_write[i]}/{current_time_str}.jpg"
    #             cv2.imwrite(img_path, frame)
    #             grabbed, frame = video_prev.read()
    #         grabbed, frame = video.read()
    #         while counter < snippet_duration and grabbed:
    #             counter += 1
    #             current_time_str = (current_time + datetime.timedelta(0, (-snippet_duration + counter) / frames_per_real_second)).strftime("%d_%m_%Y_%H_%M_%S_%f")[:-3]
    #             img_path = f"{path_write_frames_folder}/{folders_to_write[i]}/{current_time_str}.jpg"
    #             cv2.imwrite(img_path, frame)
    #             # cv2.imwrite(os.path.join(path_write_frames_folder, folders_to_write[i], f"{start_time+datetime.timedelta(0,(count - 2400 + counter)/time_per_frame)}.jpg"), frame)
    #             grabbed, frame = video.read()
    #             continue
    #     else:
    #         # print("x")
    #         video.set(cv2.CAP_PROP_POS_FRAMES, frame_location_start - snippet_duration - 1)
    #         counter = -1
    #         grabbed, frame = video.read()
    #         while counter < backwards_frames:
    #             counter += 1
    #             current_time_str = (current_time + datetime.timedelta(0, (-snippet_duration + counter) / frames_per_real_second)).strftime("%d_%m_%Y_%H_%M_%S_%f")[:-3]
    #             img_path = f"{path_write_frames_folder}/{folders_to_write[i]}/{current_time_str}.jpg"
    #             cv2.imwrite(img_path, frame)
    #             # cv2.imwrite(os.path.join(path_write_frames_folder, folders_to_write[i], f"{start_time+datetime.timedelta(0,(count - 2400 + counter)/time_per_frame)}.jpg"), frame)
    #             grabbed, frame = video.read()
    #         break
    #     break
    # return